In [ ]:
!pip install pandas.series.replace

ERROR: Could not find a version that satisfies the requirement pandas.series.replace (from versions: none)
ERROR: No matching distribution found for pandas.series.replace


In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv('Dataset.csv')

In [2]:
df['Building_Name']= np.array(range(len(df)))

In [3]:
df

,Unnamed: 0,Address,Name,Building_Name
0,0,en description YYZ1 1 1 Lbs 01 23 DT03 Leslie...,Leslie Walsh,0
1,1,en description CANADA POSTES POST 12 Expedite...,BRITTANY DHAMALIE,1
2,2,en description YYZ1 0 4 Lbs 01 23 DT03 Daniel...,Daniel Trigiani,2
3,3,en description 1 23 22 5 57 PM Production She...,Alexandra Sukhareva,3
4,4,en description YYZ1 1 0 Lbs 01 23 DT03 Don Ho...,Don Ho mes,4
...,...,...,...,...
78,78,en description YYZ1 0 3 Lbs 01 23 DT03 David ...,David Metcalfe,78
79,79,fr description 2 CANADA POSTES POST CANADA MC...,ERICA MCLACHLAN,79
80,80,en description te 327 225 EL grctu YYZ1 5 0 L...,Charlie Zheng,80
81,81,en description N708 Jo IPurolator Purolator E...,Julie Garcia,81


In [4]:
df.loc[:,'Building_Name'] = 'Vu Condos'
df.loc[:,'City'] = 'Toronto'
df.loc[:,'Zip Code'] = 'M5A 2A5'
df.loc[:,'State'] = 'Ontario'
df.loc[:,'Country'] = 'Canada'
df.loc[:,'Street_Name'] = 'George Street'
df.loc[:,'Unit_Number'] = ''

df


,Unnamed: 0,Address,Name,Building_Name,City,Zip Code,State,Country,Street_Name,Unit_Number
0,0,en description YYZ1 1 1 Lbs 01 23 DT03 Leslie...,Leslie Walsh,Vu Condos,Toronto,M5A 2A5,Ontario,Canada,George Street,
1,1,en description CANADA POSTES POST 12 Expedite...,BRITTANY DHAMALIE,Vu Condos,Toronto,M5A 2A5,Ontario,Canada,George Street,
2,2,en description YYZ1 0 4 Lbs 01 23 DT03 Daniel...,Daniel Trigiani,Vu Condos,Toronto,M5A 2A5,Ontario,Canada,George Street,
3,3,en description 1 23 22 5 57 PM Production She...,Alexandra Sukhareva,Vu Condos,Toronto,M5A 2A5,Ontario,Canada,George Street,
4,4,en description YYZ1 1 0 Lbs 01 23 DT03 Don Ho...,Don Ho mes,Vu Condos,Toronto,M5A 2A5,Ontario,Canada,George Street,
...,...,...,...,...,...,...,...,...,...,...
78,78,en description YYZ1 0 3 Lbs 01 23 DT03 David ...,David Metcalfe,Vu Condos,Toronto,M5A 2A5,Ontario,Canada,George Street,
79,79,fr description 2 CANADA POSTES POST CANADA MC...,ERICA MCLACHLAN,Vu Condos,Toronto,M5A 2A5,Ontario,Canada,George Street,
80,80,en description te 327 225 EL grctu YYZ1 5 0 L...,Charlie Zheng,Vu Condos,Toronto,M5A 2A5,Ontario,Canada,George Street,
81,81,en description N708 Jo IPurolator Purolator E...,Julie Garcia,Vu Condos,Toronto,M5A 2A5,Ontario,Canada,George Street,


In [5]:
df.rename(columns={'Name':'Recipient'}, inplace=True)





In [6]:
def massage_data(address):
    '''Pre process address string to remove new line characters, add comma punctuations etc.'''
    cleansed_address1=re.sub(r'(,)(?!\s)',', ',address)
    cleansed_address2=re.sub(r'(\\n)',', ',cleansed_address1)
    cleansed_address3=re.sub(r'(?!\s)(-)(?!\s)',' - ',cleansed_address2)
    cleansed_address=re.sub(r'\.','',cleansed_address3)
    return cleansed_address

In [7]:
def get_address_span(address=None,address_component=None,label=None):
    '''Search for specified address component and get the span.
    Eg: get_address_span(address="221 B, Baker Street, London",address_component="221",label="BUILDING_NO") would return (0,2,"BUILDING_NO")'''

    if pd.isna(address_component) or str(address_component)=='nan':
        pass
    else:
        address_component1=re.sub('\.','',address_component)
        address_component2=re.sub(r'(?!\s)(-)(?!\s)',' - ',address_component1)
        span=re.search('\\b(?:'+address_component2+')\\b',address)
        if span!=None:
          return (span.start(),span.end(),label)
        else:
          return(None)


In [8]:
def extend_list(entity_list,entity):
    if pd.isna(entity):
        return entity_list
    else:
        entity_list.append(entity)
        return entity_list

In [9]:
def create_entity_spans(df,tag_list):
    '''Create entity spans for training/test datasets'''
    df['Address']=df['Address'].apply(lambda x: massage_data(x))
    #df["BuildingTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Building_Name'],label='BUILDING_NAME'),axis=1)
    #df["BuildingNoTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Building_Number'],label='BUILDING_NO'),axis=1)
    df["RecipientTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Recipient'],label='RECIPIENT'),axis=1)
    df["StreetNameTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Street_Name'],label='STREET_NAME'),axis=1)
    #df["ZipCodeTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Zip_Code'],label='ZIP_CODE'),axis=1)
    df["CityTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['City'],label='CITY'),axis=1)
    df["CountryTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['Country'],label='COUNTRY'),axis=1)
    df["StateTag"]=df.apply(lambda row:get_address_span(address=row['Address'],address_component=row['State'],label='STATE'),axis=1)
    df['EmptySpan']=df.apply(lambda x: [], axis=1)

    for i in tag_list:
        df['EntitySpans']=df.apply(lambda row: extend_list(row['EmptySpan'],row[i]),axis=1)
        df['EntitySpans']=df[['EntitySpans','Address']].apply(lambda x: (x[1], x[0]),axis=1)
    return df['EntitySpans']

In [10]:
def get_doc_bin(training_data,nlp):
    '''Create DocBin object for building training/test corpus'''
    # the DocBin will store the example documents
    db = DocBin()
    for text, annotations in training_data:
        doc = nlp(text) #Construct a Doc object
        ents = []
        for start, end, label in annotations:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
    return db

In [11]:
import spacy
import re
from spacy.tokens import DocBin

In [14]:
#Load blank English model. This is needed for initializing a Document object for our training/test set.
nlp = spacy.blank("en")

#Define custom entity tag list
tag_list=["RecipientTag","StreetNameTag","CityTag","StateTag","CountryTag"]

###### Training dataset prep ###########
# Read the training dataset into pandas
df_train=df

# Get entity spans
df_entity_spans= create_entity_spans(df_train.astype(str),tag_list)
training_data= df_entity_spans.values.tolist()

# Get & Persist DocBin to disk
doc_bin_train= get_doc_bin(training_data,nlp)
doc_bin_train.to_disk("train.spacy")

In [30]:
x=get_address_span(df['Address'][0],'Leslie Walsh','Name')
print(x)

(40, 52, 'Name')


In [83]:
!pip install spacy==3.0.6

     |████████████████████████████████| 12.8 MB 4.3 MB/s 
     |████████████████████████████████| 628 kB 33.5 MB/s 
     |████████████████████████████████| 9.1 MB 34.8 MB/s 
     |████████████████████████████████| 42 kB 728 kB/s 
     |████████████████████████████████| 451 kB 46.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
